In [2]:
import numpy as np
import pickle
import utils
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/home/davidhou8791/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
emoji_vectors = pickle.load(open('data/emoji_vectors.p', 'rb'))
moby_dick_vectors = pickle.load(open('data/moby_dick_vectors.p', 'rb'))
moby_dick_sents = pickle.load(open('data/moby_dick_sents.p', 'rb'))

emoji_embedding = np.array([v for v in emoji_vectors.values()])

In [11]:
tf.reset_default_graph()
sess.close()
sess = tf.InteractiveSession()

In [12]:
batch_size = 128
nodes = 300
embed_size = 300
x_seq_length = 32

inputs = tf.placeholder(tf.float32, (None, x_seq_length, embed_size), 'inputs')
input_mean = tf.nn.l2_normalize(tf.reduce_mean(inputs, axis=1), axis=1, name='input_mean')

output_embedding = tf.constant(emoji_embedding, name='output_embedding')

with tf.name_scope('network'):
    lstm_encoder = tf.contrib.rnn.LSTMCell(nodes, name='lstm_encoder')
    _, encoding = tf.nn.dynamic_rnn(lstm_encoder, inputs=inputs, dtype=tf.float32)
    
    lstm_decoder = tf.contrib.rnn.LSTMCell(nodes, name='lstm_decoder')
    lstm_outputs, _ = tf.nn.dynamic_rnn(lstm_decoder, inputs=inputs, initial_state=encoding, dtype=tf.float32)

    logits = tf.layers.dense(lstm_outputs, units=len(emoji_vectors), activation='softmax', name='dense') 
    outputs = utils.matmul3d(logits, output_embedding)

    output_mean = tf.nn.l2_normalize(tf.reduce_mean(outputs, axis=1), axis=1)

with tf.name_scope("optimization"):
    loss = tf.losses.cosine_distance(input_mean, output_mean, axis=1)
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
tf.summary.scalar('loss', loss)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('models/seq2seq_moby_dick/1', sess.graph)

In [15]:
train_sents, test_sents = train_test_split(moby_dick_sents, random_state=42)

In [16]:
X_train = [[moby_dick_vectors[w] for w in s] for s in train_sents]
X_test = [[moby_dick_vectors[w] for w in s] for s in test_sents]

In [19]:
def batch_generator(X, batch_size):
    i = 0
    while i < len(X):
        yield X[i:i+batch_size]
        i += batch_size

In [19]:
sess.run(tf.global_variables_initializer())
epochs = 100
start = time.time()

for i in range(epochs):
    t = time.time()
    losses = []
    for X in utils.batch_generator(X_train, batch_size):
        _, l, summary = sess.run([optimizer, loss, merged], feed_dict={inputs:X})
        losses.append(l)
    writer.add_summary(summary, global_step=i)
#     if l < .0005:
#         print('Epoch {:3} Loss: {:>6.3f} Epoch duration: {:>6.3f}s'.format(i, l, time.time() - t))
#         break
#     elif not i%10:
    print('Epoch {:3} Average Loss: {:>6.3f} Epoch duration: {:>6.3f}s'.format(i, np.mean(losses, axis=-1), time.time() - t))

saver = tf.train.Saver()
saver.save(sess, 'models/seq2seq_moby_dick/model1')
print('Total training time:', time.time()-start)

predictions = []
losses = []
emoji_keys = list(emoji_vectors.keys())
for x in tqdm(X_test):
    lo, l = sess.run([logits, loss], feed_dict={inputs:np.array(x).reshape(-1, 32, 300)})
    pred = np.argmax(lo, axis=2).reshape(32,)
    predictions.append([emoji_keys[i] for i in pred])
    losses.append(l)

print('Average test loss:', np.mean(losses, axis=-1))
print()

for i in range(10):
    print('Test sentence:', ' '.join(w for w in test_sents[i] if w))
    print('Prediction:', set(predictions[i]))
    print('Cosine distance:', losses[i])
    print()

Epoch   0 Average Loss:  0.406 Epoch duration: 48.220s
Epoch   1 Average Loss:  0.380 Epoch duration: 47.975s
Epoch   2 Average Loss:  0.379 Epoch duration: 47.631s
Epoch   3 Average Loss:  0.371 Epoch duration: 47.624s
Epoch   4 Average Loss:  0.358 Epoch duration: 48.705s
Epoch   5 Average Loss:  0.350 Epoch duration: 47.596s
Epoch   6 Average Loss:  0.343 Epoch duration: 48.086s
Epoch   7 Average Loss:  0.339 Epoch duration: 47.785s
Epoch   8 Average Loss:  0.335 Epoch duration: 47.947s
Epoch   9 Average Loss:  0.332 Epoch duration: 47.948s
Epoch  10 Average Loss:  0.330 Epoch duration: 47.780s
Epoch  11 Average Loss:  0.328 Epoch duration: 48.719s
Epoch  12 Average Loss:  0.325 Epoch duration: 48.195s
Epoch  13 Average Loss:  0.322 Epoch duration: 47.696s
Epoch  14 Average Loss:  0.320 Epoch duration: 47.751s
Epoch  15 Average Loss:  0.318 Epoch duration: 47.605s
Epoch  16 Average Loss:  0.317 Epoch duration: 47.565s
Epoch  17 Average Loss:  0.315 Epoch duration: 48.736s
Epoch  18 

  0%|          | 0/2515 [00:00<?, ?it/s]

Epoch  99 Average Loss:  0.292 Epoch duration: 47.046s
Total training time: 4800.203766345978


100%|██████████| 2515/2515 [00:48<00:00, 50.67it/s]

Average test loss: 0.30878183

Test sentence: an old pike head sir there were seams dents in it
Prediction: {'🈁', '4', '🤕'}
Cosine distance: 0.22518182

Test sentence: this one poor hunt then the best lance out all surely he will not hang back when every foremast hand has clutched whetstone
Prediction: {'🔂', '🆙', '🈁', '4', '⚜', '😰'}
Cosine distance: 0.12840074

Test sentence: drop them over fore aft
Prediction: {'⚜', '4'}
Cosine distance: 0.3641491

Test sentence: in the infancy the first settlement the emigrants were several times saved from starvation by the benevolent biscuit the whale ship luckily dropping an anchor in their waters
Prediction: {'🏤', '🤒', '🍠', '♌', '🈴', '🆘', '⛵', '🈁', '🕦', '💯', '4', '🍞'}
Cosine distance: 0.16920751

Test sentence: mighty whales which swim in sea water have sea oil swimming in them
Prediction: {'⛽', '♓', '⛵', '🥘', '4'}
Cosine distance: 0.17872876

Test sentence: round round the fish s back pinioned in the turns upon turns in which during the past nig

In [11]:
predictions = []
losses = []
emoji_keys = list(emoji_vectors.keys())
for x in tqdm(X_test):
    lo, l = sess.run([logits, loss], feed_dict={inputs:np.array(x).reshape(-1, 32, 300)})
    pred = np.argmax(lo, axis=2).reshape(32,)
    predictions.append([emoji_keys[i] for i in pred])
    losses.append(l)

100%|██████████| 2515/2515 [00:29<00:00, 86.02it/s]


In [12]:
for i in range(10):
    print('Test sentence:', ' '.join(w for w in test_sents[i] if w))
    print('Prediction:', set(predictions[i]))
    print('Cosine distance:', losses[i])
    print()

Test sentence: an old pike head sir there were seams dents in it
Prediction: {'🤕', '🈁', '♈', '🐓'}
Cosine distance: 0.2244193

Test sentence: this one poor hunt then the best lance out all surely he will not hang back when every foremast hand has clutched whetstone
Prediction: {'⚛', '🆙', '🈁', '🙏', '🔂', '⚜', '♈', '🐓'}
Cosine distance: 0.13012469

Test sentence: drop them over fore aft
Prediction: {'⚜', '♈', '🐓', '🆙'}
Cosine distance: 0.36446148

Test sentence: in the infancy the first settlement the emigrants were several times saved from starvation by the benevolent biscuit the whale ship luckily dropping an anchor in their waters
Prediction: {'♏', '🛳', '🆙', '🍠', '🈁', '🈴', '♈', '⚰', '⛵', '〽', '🐓', '🔖'}
Cosine distance: 0.16519237

Test sentence: mighty whales which swim in sea water have sea oil swimming in them
Prediction: {'🐋', '♈', '⛽', '🤼', '🌊', '🐓'}
Cosine distance: 0.14461929

Test sentence: round round the fish s back pinioned in the turns upon turns in which during the past nigh